In [27]:
import pandas as pd

In [28]:
df_full = pd.read_csv('D:\\GITHUB\\Data_CEP\\Exploration\\dataframe_expanded.csv')
df_with_id = pd.read_csv('D:\\GITHUB\\Data_CEP\\Collection\\Data\\Wyscout\\full_dataframe_concatenated.csv')
df_selected = pd.read_csv('D:\\GITHUB\\Data_CEP\\Pre_Processing\\combined_selected_rows.csv')

C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_7372\2145914929.py:3: DtypeWarning: Columns (21,25,26,29,32,36,38,39,40,43,44,45,46,49,50,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_selected = pd.read_csv('D:\\GITHUB\\Data_CEP\\Pre_Processing\\combined_selected_rows.csv')


In [34]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 250
print(df_with_id[df_with_id['primary_type'] == 'duel'])


          event_id  match_id match_period  minute  second match_timestamp  \
5       1723218229   5476240           1H       0       9    00:00:09.331   
6       1723218259   5476240           1H       0      10    00:00:10.793   
27      1723218252   5476240           1H       1      39    00:01:39.978   
28      1723218287   5476240           1H       1      39    00:01:39.987   
37      1723218272   5476240           1H       2       7    00:02:07.103   
...            ...       ...          ...     ...     ...             ...   
495867  2140029980   5476539           2H      95      22    01:35:22.562   
495871  2140029874   5476539           2H      95      53    01:35:53.039   
495872  2140029983   5476539           2H      95      53    01:35:53.039   
495873  2140029877   5476539           2H      95      54    01:35:54.938   
495874  2140029984   5476539           2H      95      54    01:35:54.938   

        video_timestamp primary_type  \
5             10.331259         due

In [35]:
merged_df = pd.merge(df_selected, df_with_id[['event_id', 'related_duel_id_ground_duel']], on='event_id', how='left', suffixes=('_dfselected', '_dfid'))

In [36]:
df_selected['related_duel_id_ground_duel'] = merged_df['related_duel_id_ground_duel_dfid']

In [41]:
merged_df_full = pd.merge(df_full, df_with_id[['event_id', 'related_duel_id_ground_duel']], on='event_id', how='left', suffixes=('_dffull', '_dfid'))

In [42]:
df_full['related_duel_id_ground_duel'] = merged_df_full['related_duel_id_ground_duel_dfid']


In [43]:
df_full.columns

Index(['Unnamed: 0', 'event_id', 'match_id', 'match_period', 'minute',
       'second', 'match_timestamp', 'video_timestamp', 'primary_type',
       'secondary_types',
       ...
       'shot', 'head_shot', 'whistle', 'shot_after_free_kick', 'acceleration',
       'penalty_foul', 'penalty_goal', 'penalty_conceded_goal', 'third_assist',
       'penalty_save'],
      dtype='object', length=142)

In [45]:
df_selected['video_timestamp_previous'] = None
df_selected['video_timestamp_next'] = None
df_selected['primary_type_next'] = None
df_selected['primary_type_previous'] = None
df_selected['video_timestamp_2_events_previous'] = None
df_selected['video_timestamp_2_events_next'] = None

 
# Create dictionaries for quick look-up
index_to_timestamp = dict(zip(df_full.index, df_full['video_timestamp']))
index_to_primary = dict(zip(df_full.index, df_full['primary_type']))
index_to_loose_ball = dict(zip(df_full.index, df_full['loose_ball_duel']))

count_rare = 0
# Populate the new columns with previous and next timestamps
for index, row in df_selected.iterrows():
    event_id = row['event_id']
    primary_type = row['primary_type']
    loose_ball_duel = row['loose_ball_duel']
    current_index = df_full.index[df_full['event_id'] == event_id][0]
    #print(current_index)
    if (primary_type == 'duel') & (loose_ball_duel != 1):
        # Get related duel IDs from df_full
        related_duel_aerial = df_full[df_full['event_id'] == event_id]['related_duel_id_aerial_duel'].values[0]
        index_related_duel_aerial = df_full.index[df_full['event_id'] == related_duel_aerial]
        related_duel_ground = df_full[df_full['event_id'] == event_id]['related_duel_id_ground_duel'].values[0]
        index_related_duel_ground = df_full.index[df_full['event_id'] == related_duel_ground]

        if index_related_duel_aerial == current_index - 1 or index_related_duel_ground == current_index - 1:
            df_selected.at[index, 'video_timestamp_previous'] = index_to_timestamp.get(current_index - 2, None)
            df_selected.at[index, 'primary_type_previous'] = index_to_primary.get(current_index -2, None)
            if ((index_to_primary.get(current_index-2, None) == 'duel') & (index_to_loose_ball.get(current_index-2, None) != 1)):
                df_selected.at[index, 'video_timestamp_2_events_previous'] = index_to_timestamp.get(current_index-4, None)
            else: 
                df_selected.at[index, 'video_timestamp_2_events_previous'] = index_to_timestamp.get(current_index-3, None)
            df_selected.at[index, 'primary_type_next'] = index_to_primary.get(current_index + 1)
            df_selected.at[index, 'video_timestamp_next'] = index_to_timestamp.get(current_index + 1, None)
            if ((index_to_primary.get(current_index+1, None) == 'duel') & (index_to_loose_ball.get(current_index+1, None) !=1)):
                df_selected.at[index, 'video_timestamp_2_events_next'] = index_to_timestamp.get(current_index+3, None)
            else: 
                df_selected.at[index, 'video_timestamp_2_events_next'] = index_to_timestamp.get(current_index+2, None)
    
        elif index_related_duel_aerial == current_index + 1 or index_related_duel_ground == current_index + 1:
            df_selected.at[index, 'video_timestamp_next'] = index_to_timestamp.get(current_index + 2, None)
            df_selected.at[index, 'primary_type_next'] = index_to_primary.get(current_index + 2, None)
            if ((index_to_primary.get(current_index+2, None) == 'duel') & (index_to_loose_ball.get(current_index+2, None) !=1)):
                df_selected.at[index, 'video_timestamp_2_events_next'] = index_to_timestamp.get(current_index+4, None)
            else: 
                df_selected.at[index, 'video_timestamp_2_events_next'] = index_to_timestamp.get(current_index+3, None)
            df_selected.at[index, 'primary_type_previous'] = index_to_primary.get(current_index-1, None)
            df_selected.at[index, 'video_timestamp_previous'] = index_to_timestamp.get(current_index - 1, None)
            if ((index_to_primary.get(current_index-1, None) == 'duel') & (index_to_loose_ball.get(current_index-1, None) != 1)):
                df_selected.at[index, 'video_timestamp_2_events_previous'] = index_to_timestamp.get(current_index-3, None)
            else: 
                df_selected.at[index, 'video_timestamp_2_events_previous'] = index_to_timestamp.get(current_index-2, None)

        else: 
            count_rare += 1
    else:
        df_selected.at[index, 'video_timestamp_previous'] = index_to_timestamp.get(current_index - 1, None)
        df_selected.at[index, 'primary_type_previous'] = index_to_primary.get(current_index-1, None)
        if ((index_to_primary.get(current_index -1, None) == 'duel') & (index_to_loose_ball.get(current_index -1, None) != 1)):
            df_selected.at[index, 'video_timestamp_2_events_previous'] = index_to_timestamp.get(current_index-3, None) 
        else:
            df_selected.at[index, 'video_timestamp_2_events_previous'] = index_to_timestamp.get(current_index-2, None)
        df_selected.at[index, 'video_timestamp_next'] = index_to_timestamp.get(current_index + 1, None)
        df_selected.at[index, 'primary_type_next'] = index_to_primary.get(current_index + 1, None)
        if ((index_to_primary.get(current_index+1, None) == 'duel') & (index_to_loose_ball.get(current_index +1, None) != 1)):
            df_selected.at[index, 'video_timestamp_2_events_next'] = index_to_timestamp.get(current_index+3, None)
        else:
            df_selected.at[index, 'video_timestamp_2_events_next'] = index_to_timestamp.get(current_index+2, None)





C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_7372\3856934372.py:29: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if index_related_duel_aerial == current_index - 1 or index_related_duel_ground == current_index - 1:
C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_7372\3856934372.py:43: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  elif index_related_duel_aerial == current_index + 1 or index_related_duel_ground == current_index + 1:


In [46]:
print(count_rare)

57


In [47]:
nan_count_timestamp_previous_na = df_selected[df_selected['video_timestamp_previous'].isna()]
print(nan_count_timestamp_previous_na)
nan_count_timestamp_next_na = df_selected[df_selected['video_timestamp_next'].isna()]
print(nan_count_timestamp_next_na)
# nan_count_begin_clip = df[df['begin_clip_timestamp'].isna()]
# print(nan_count_begin_clip['primary_type'].value_counts())

      Unnamed: 0    event_id  match_id match_period  minute  second  \
2960      230885  1866521883   5476376           1H      31       5   
2983      264441  1877223804   5476396           1H      46       4   
3008      262959  1874453502   5476395           2H      59      41   
3072       50311  1736979358   5476269           2H      72       8   
3098      463727  2121410583   5476516           2H      94      58   
3254      219877  1863757081   5476369           2H      86      23   
4055       34083  1744222764   5476260           1H      14      35   
4096      397822  2088097076   5476476           1H       8       6   
4217      458306  2129145058   5476513           2H      64      50   
4291      219674  1863756931   5476369           2H      73      30   
4474       79155  1767071144   5476287           1H       6      55   
4480      199368  1834874167   5476357           2H      94      43   
4501      219673  1863755633   5476369           2H      73      30   
4552  

In [48]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 200
df_selected['timestamp_previous_difference'] = df_selected['video_timestamp'] - df_selected['video_timestamp_previous']
df_selected['timestamp_next_difference'] = df_selected['video_timestamp_next'] - df_selected['video_timestamp']
#print(df_selected['timestamp_previous_difference'])
count = df_selected[(df_selected['timestamp_previous_difference'] < 0.00)]
print(len(count))
count_next = df_selected[(df_selected['timestamp_next_difference'] < 0.00)]
print(len(count_next))
count_2 = df_selected[(df_selected['timestamp_previous_difference'] > 10.00) & (df_selected['timestamp_previous_difference'] < 30.00)]
print(len(count_2))
count_next_2 = df_selected[(df_selected['timestamp_next_difference'] > 10.00) & (df_selected['timestamp_next_difference'] < 20.00)]
print(len(count_next_2))

count_3 = df_selected[df_selected['timestamp_previous_difference'] > 10.00]['primary_type_previous'].value_counts()
print(count_3)
count_next_3 = df_selected[df_selected['timestamp_next_difference'] > 10.00]['primary_type_next'].value_counts()
print(count_next_3)

count_4 = df_selected[df_selected['timestamp_previous_difference'] > 10.00].groupby(['primary_type_previous', 'primary_type']).size().reset_index(name='counts')
print(count_4)
count_next_4 = df_selected[df_selected['timestamp_next_difference'] > 10.00].groupby(['primary_type_next', 'primary_type']).size().reset_index(name='counts')
print(count_next_4)

7
16
95
192
shot_against         41
touch                33
game_interruption    15
pass                  9
interception          8
goalkeeper_exit       5
duel                  4
free_kick             3
goal_kick             2
throw_in              2
corner                1
clearance             1
acceleration          1
fairplay              1
Name: primary_type_previous, dtype: int64
game_interruption    125
pass                 116
duel                   8
interception           3
infraction             2
touch                  2
Name: primary_type_next, dtype: int64
   primary_type_previous  primary_type  counts
0           acceleration          pass       1
1              clearance          pass       1
2                 corner          duel       1
3                   duel          duel       1
4                   duel          pass       2
5                   duel         touch       1
6               fairplay          pass       1
7              free_kick          duel       1

In [49]:
df_selected['begin_clip_timestamp'] = None
df_selected['end_clip_timestamp'] = None

df_selected['begin_clip_timestamp'] = df_selected['video_timestamp'] - 1.25*df_selected['timestamp_previous_difference']
df_selected['end_clip_timestamp'] = df_selected['video_timestamp'] + 1.25*df_selected['timestamp_next_difference']

count_prev = 0
count_next = 0

#mask_prev = df_selected[(df_selected['begin_clip_timestamp'] < df_selected['video_timestamp_2_events_previous'])]
mask_prev = df_selected['begin_clip_timestamp'] < df_selected['video_timestamp_2_events_previous']
#print(mask_prev)
count_prev = mask_prev.sum()
mask_next = df_selected['end_clip_timestamp'] > df_selected['video_timestamp_2_events_next']
count_next = mask_next.sum()
print(count_prev, count_next)

# Update begin_clip_timestamp based on condition
mask_prev = df_selected['begin_clip_timestamp'] < df_selected['video_timestamp_2_events_previous']
df_selected.loc[mask_prev, 'begin_clip_timestamp'] = df_selected.loc[mask_prev, 'video_timestamp_2_events_previous']
# Update end_clip_timestamp based on condition
mask_next = df_selected['end_clip_timestamp'] > df_selected['video_timestamp_2_events_next']
df_selected.loc[mask_next, 'end_clip_timestamp'] = df_selected.loc[mask_next, 'video_timestamp_2_events_next']



1609 2109


In [ ]:
mask_prev

In [50]:
#Removing the rows where the match changes
print(len(df_selected))
df_selected = df_selected[~(df_selected['timestamp_previous_difference'] < 0.00)]
df_selected = df_selected[~(df_selected['timestamp_next_difference'] > 100.00)]
df_selected = df_selected[~(df_selected['timestamp_next_difference'] < 0.00)]
print(len(df_selected))



#Change timestamp start for more than 10 sec with shot_against, touch, or game_interruption as 
count_3 = df_selected[df_selected['timestamp_previous_difference'] > 10.00]['primary_type_previous'].value_counts()
condition = (
    (df_selected['timestamp_previous_difference'] > 10.00) &
    (df_selected['primary_type_previous'].isin(['shot_against', 'game_interruption']))
)
# Apply the condition to update the begin_clip_timestamp
df_selected.loc[condition, 'begin_clip_timestamp'] = df_selected['video_timestamp'] - 0.5

condition = (
    (df_selected['timestamp_next_difference'] > 10.00) &
    (df_selected['primary_type_next'].isin(['game_interruption']))
)
df_selected.loc[condition, 'end_clip_timestamp'] = df_selected['video_timestamp'] + 5.00

condition = (
    ((df_selected['video_timestamp'] - df_selected['begin_clip_timestamp']) > 30) |
    ((df_selected['end_clip_timestamp'] - df_selected['video_timestamp']) > 30)
)
print(len(df_selected))
df_selected_test = df_selected[~condition]
print(len(df_selected_test))


26650
26624
26624
26611


In [51]:
# Calculate the length of the clip
df_selected_test['length_clip'] = df_selected_test['end_clip_timestamp'] - df_selected_test['begin_clip_timestamp']

# Ensure the new column has numeric values (this should be redundant if the columns are numeric)
df_selected_test['length_clip'] = pd.to_numeric(df_selected_test['length_clip'], errors='coerce')

# Print the mean of the length_clip column
print(df_selected_test['length_clip'].mean())

0.18510876267276413


C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_7372\356824822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_test['length_clip'] = df_selected_test['end_clip_timestamp'] - df_selected_test['begin_clip_timestamp']
C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_7372\356824822.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_test['length_clip'] = pd.to_numeric(df_selected_test['length_clip'], errors='coerce')


In [52]:
df_selected_test = df_selected_test[~(df_selected_test['length_clip']<0)]
print(df_selected_test['length_clip'].mean())
df_selected_test = df_selected_test[~(df_selected_test['length_clip']>60)]
print(df_selected_test['length_clip'].mean())

5.290304224255557
5.290304224255557


In [53]:
print(len(df_selected_test))

26588


In [54]:
df_selected_test.to_csv('D:\\GITHUB\\Data_CEP\\Pre_Processing\\Slicing\\selected_clips.csv')